In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.preprocessing import image


In [ ]:
#Upload the  directories of train,test,validation data set
train_data_dir = ''
valid_data_dir = ''
test_data_dir = ''


In [ ]:
img_width, img_height = 299, 299
batch_size = 32
num_epochs = 10


In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1.0/255)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(valid_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='binary',
                                                  shuffle=False)


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=num_epochs,
          validation_data=valid_generator,
          validation_steps=valid_generator.samples // batch_size)


In [ ]:
scores = model.evaluate(test_generator)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


In [ ]:
def predict_osteoarthritis(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image
    prediction = model.predict(img)
    probability = prediction[0][0]
    if probability < 0.5:
        print(f"The probability of having osteoarthritis is {probability:.2f}. Osteoarthritis is not detected.")
    else:
        print(f"The probability of having osteoarthritis is {probability:.2f}. Osteoarthritis is detected.")
